## Руководитель агентов

В [предыдущем примере](multi-agent-collaboration.ipynb) вы можете ознакомиться с автоматической маршрутизацией сообщений в зависимости от результата работы первичного агента-исследователя.

Кроме такого подхода вы можете использовать LLM для организации работы агентов.

Пример ниже показывает как создать группу агентов с LLM в роли руководителя, который помогает распределять задачи.

![Схема](./img/supervisor-diagram.png)

Для упрощения кода каждой из вершин, в которых распологаются агенты, используется класс `AgentExecutor` библиотеки LangChain. Таким образом этот и другие сложные примеры показывают различные подходы к проектированию с помощью LangGraph. Комбинируйте эти подходы с другими базовыми решениями, для более эффективного решения своих задач. 

Перед началом разработки подготовьте среду:

In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain langchain_openai langchain_experimental langsmith pandas

In [3]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

## Создание инструментов

Для этого примера вам потребуется два агента. Первый будет выполнять исследование с помощью запросов к поисковой системе, а второй будет отрисовывать графики.

Задайте инструменты, которые будут использовать каждый из агентов:

In [ ]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)

# Функция выполняет код локально, что может быть небезсопасно
python_repl_tool = PythonREPLTool()

## Создание вспомогательных функций

Создайте вспомогательную функцию, которая упростит добавление вершин с агентами-исполнителями:

In [5]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Каждой вершине с исполнителем присваивается имя и дается доступ
    # к определенным инструментам
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

Также определите функцию, которая будет представлять вершины графа. Функция будет преобразовывать ответ агента в сообщения от человека (`HumanMessage`). Это нужно для работы с глобальным состоянием графа.

In [6]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

### Создайте руководителя агентов

Руководитель вызывает функцию, чтобы определенить следующую вершину-исполнителя или завершить обработку.

In [12]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Researcher", "Coder"]
system_prompt = (
    "Вы - руководитель, задача которого заключается в управлении разговором между"
    " следующими сотрудниками: {members}. Учитывая следующий запрос пользователя,"
    " ответьте, какой сотрудник должен действовать далее. Каждый сотрудник будет выполнять"
    " задачу и отвечать своими результатами и статусом. Когда работа будет завершена,"
    " ответьте FINISH."
)
# Руководитель агентов — это LLM-вершина графа. Руководитель определяет
# какому из агентов передать задачу и принимает решение о ее выполнении
options = ["FINISH"] + members
# Использование openai-функций упрощает обработку вывода
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Исходя из приведенного выше разговора, кто должен действовать дальше?"
            " Или мы должны завершить? Выберите один из: {options}",
        )
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4-1106-preview")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

## Соберите граф

Теперь вы можете собрать граф.
Для этого определите состояние агента и вершины-исполнители. Для определения вершин используйте созданную вспомогательную функцию.

In [13]:
import functools
import operator
from typing import Sequence, TypedDict

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph


# Состояние агента передается на вход каждой из вершин графа
class AgentState(TypedDict):
    # Аннотация указывает графу, что новые сообщения нужно всегда
    # добавлять к текущим состояниям
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # Поле 'next' указывает, что делать дальше
    next: str


research_agent = create_agent(llm, [tavily_tool], "Ты аналитик, который ищет информацию и обрабатывает результаты")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# NOTE: НА ЭТОМ ЭТАПЕ ВЫПОЛНЯЕТСЯ ПРОИЗВОЛЬНЫЙ КОД. БУДЬТЕ ОСТОРОЖНЫ
code_agent = create_agent(
    llm,
    [python_repl_tool],
    "Вы можете генерировать безопасный код на Python для анализа данных и создания графиков с использованием matplotlib.",
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)

Соедините все ребра графа.

In [14]:
for member in members:
    # Исполнители должны ВСЕГДА докладывать руководителю о завершении работы
    workflow.add_edge(member, "supervisor")
# Руководитель заполняет поле "next" в состостоянии графа,
# которое переходит к другой вершине или заканчивает обработку
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Добавьте точку входа
workflow.set_entry_point("supervisor")

graph = workflow.compile()

## Запустите граф

Теперь, когда граф собран, вы можете запустить его и посмотреть, как он работает.

In [23]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Напиши на python функцию для вычисления факториала и найди факториал числа 15.")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Coder'}}
----
{'Coder': {'messages': [HumanMessage(content='Факториал числа 15 равен 1,307,674,368,000.', name='Coder')]}}
----
{'supervisor': {'next': 'Coder'}}
----
{'Coder': {'messages': [HumanMessage(content='Функция для вычисления факториала числа была успешно выполнена, и факториал числа 15 равен 1,307,674,368,000.', name='Coder')]}}
----
{'supervisor': {'next': 'FINISH'}}
----


In [22]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Напиши краткий отчет о том, за что Илон Маск подал в суд на OpenAI")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='Илон Маск подал иск против OpenAI и её генерального директора Сэма Альтмана. Основные обвинения касаются нарушения договора, при котором компания, по словам Маска, отошла от своей первоначальной миссии как некоммерческой организации. Маск утверждает, что OpenAI ушла от принципа открытости, партнерствуя с Microsoft на 13 миллиардов долларов и сохраняя в тайне код своих новейших продуктов в области генеративного искусства.\n\nOpenAI была соучредителем Илона Маска в 2015 году, но с тех пор он покинул компанию и основал свою собственную компанию в области искусственного интеллекта, xAI. В иске упоминается, что Microsoft, которая вложила миллиарды долларов в OpenAI и имеет тесное партнерство с стартапом, не является ответчиком, но упоминается в жалобе 68 раз.\n\nМаск обвиняет OpenAI и Сэма Альтмана в том, что они ставят прибыль и коммерческие интересы в разработке искусственного интеллекта выше об